In [ ]:
from transformers import pipeline
import transformers
import torch
# base_model = "mistralai/Mistral-7B-Instruct-v0.2"
base_model = 'HuggingFaceH4/zephyr-7b-beta'
# full_output = 'models/full_KUET_LLM_Mistral'
full_output = 'models/full_KUET_LLM_zepyhr'
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model)
# pipe = pipeline(task="text-generation", model=full_output, tokenizer=tokenizer, max_length=630)
pipe = pipeline("text-generation",
                model=full_output,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

In [ ]:
from langchain.llms import HuggingFaceTextGenInference
from langchain.llms import HuggingFaceTextGenInference
from langchain import PromptTemplate
from langchain.schema import StrOutputParser

import warnings
warnings.simplefilter('ignore')


template = """
    <s>[INST] <<SYS>>
    {role}
    <</SYS>>       
    {text} [/INST]
"""

prompt = PromptTemplate(
    input_variables = [
        "role", 
        "text"
    ],
    template = template,
)

role = "You are a KUET authority managed chatbot, help users by answering their queries about KUET."


chain = prompt | llm | StrOutputParser()

import pandas as pd
import tqdm
import warnings
warnings.filterwarnings('ignore')
dat=[]
cnt=0
df=pd.read_excel(r"data/KUET information2k20.xlsx")

for id,ques in tqdm.tqdm(zip(df['id'],df['Question'])):
    ans=chain.invoke({"role": role,"text":ques})
    dat.append({
        "id":id,
        "question":ques,
        "answer":ans
    })



In [ ]:
sa=pd.DataFrame(dat)
model_ans="model_ans_zepyhr_finetuned486_without_rag"
sa.to_excel(f"data//{model_ans}.xlsx",index=False)

In [ ]:
full_output = 'models/full_KUET_LLM_llama'
import transformers
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(full_output)
model = transformers.AutoModelForCausalLM.from_pretrained(full_output)

In [ ]:
from huggingface_hub import login

In [ ]:
login()

In [ ]:
model_name="shahidul034/KUETLLM_llama2"
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)